# 실력 긍부정

In [1]:
# 실력 긍부정 데이터 불러오기
import pandas as pd

sentence_df = pd.read_csv('./level-lable-3.csv', header=0, sep=',')
sentence_df.head(3)

,sentence,lable
0,"카오스, 아키에이지, 2012년~2014년 리그 오브 레전드, 2016년 오버워치,...",1
1,어떤 게임이든지 파고들기 시작하면 최상위권을 달성하며 엄청난 실력의 재능충 모습을 ...,1
2,게임 센스가 매우 뛰어난 편이며 화려한 플레이가 자주 나오는 편이다.,1


In [2]:
# 전처리 1
import re

sentence_df['sentence'] = sentence_df['sentence'].apply(lambda x : re.sub(pattern="[^ㄱ-ㅎ 가-힣 a-z A-Z]", repl=" ", string=str(x)))

sentence_df.head(10)

,sentence,lable
0,카오스 아키에이지 년 년 리그 오브 레전드 년 오버워치 ...,1
1,어떤 게임이든지 파고들기 시작하면 최상위권을 달성하며 엄청난 실력의 재능충 모습을 ...,1
2,게임 센스가 매우 뛰어난 편이며 화려한 플레이가 자주 나오는 편이다,1
3,이때는 잿불거인이 나타나기 전으로 우디르가 고인챔이라는 인식이 강해 천상계에서 아무...,1
4,당연히 천상계에선 원탑 우디르 유저,1
5,오버워치와 비슷한 FPS게임에 재능을 보이는 파카답게 솔로 스쿼드를 가리지 않고 엄...,1
6,롤 PD들이 파카 방송에 모여 스쿼드를 자주 하였는데 PD들 한두 명이 킬 킬...,1
7,플레이 스타일은 피지컬과 반응속도로 찍어누르는 재능충의 표본,1
8,뛰어난 로지컬과 상황 판단력으로 각을 재고 들어가는 타입이 아니라 일단 몸을 던지며...,1
9,의 상황에서도 본인의 피지컬을 신뢰하며 절대 빼지 않고 몸을 던진다,1


In [8]:
# 명사 추출 ( 안하는게 성능 더 좋을수도)

from konlpy.tag import Hannanum

hannanum = Hannanum()
i = 0
for sentence in sentence_df['sentence']:
    original = sentence
    string2noun = hannanum.nouns(original)
    noun2string = " ".join(string2noun)
    sentence_df['sentence'][i] = noun2string
    i += 1

sentence_df.head(10)

,sentence,lable
0,카오스 아키 년 년 리그 오브 레전드 년 오버워치 년 년 배틀그라운드 등 해 인기 ...,1
1,게임 시작 최상위권 달성 실력 재능충 모습 보,1
2,게임 센스 편 화 플레이 편,1
3,이때 잿불거인 전 우디르 고인챔 식 천상계 아무 우디르 우디르 승률 어마어마 수치 ...,1
4,천상계 원탑 우디르 유저,1
5,오버워치 비슷 재능 파카답게 솔 스쿼드,1
6,롤 파카 방송 스쿼드 들 한두 명 킬 멱살잡 우승 버스 기사 역할,1
7,플레이 스타일 피지컬 반응속도 찍어누르 재능충 표본,1
8,로지컬 상황 판단력 각 재고 타입 몸 공격적 포지셔닝,1
9,의 상황 본인 피지컬 신뢰 몸,1


In [9]:
# train-test set 나누기

from sklearn.model_selection import train_test_split

class_df = sentence_df['lable']
feature_df = sentence_df.drop(['lable'],axis=1, inplace=False)



X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.2, random_state=20)

X_train.shape, X_test.shape

((317, 1), (80, 1))

In [12]:
# 로지스틱 회귀 + countVectorize


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(C=10))

])

pipeline.fit(X_train['sentence'], y_train)
pred = pipeline.predict(X_test['sentence'])
pred_probs = pipeline.predict_proba(X_test['sentence'])[:,1]

print('예측 정확도는 {0:.4f}, ROC_AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

cnt = 0
for review in X_test['sentence']:
    if pred[cnt] == 1:
        sentiment = 'positive'
    else:
        sentiment = 'negative'

    cnt += 1
    if cnt == 100:
        break

    print('댓글 :{0} 는 {1}로 예상됩니다'.format(review, sentiment))

예측 정확도는 0.8125, ROC_AUC는 0.9169
댓글 :챔피언 폭도 진 같은 챔피언 보다는 베인 이나 트위치 코그모 같은 AD 하이퍼 캐리 선호 하며 그것 아니라면 케이틀린 이나 자야 같은 챔피언 골라 라인 전 에서부터 압살 하려 챔피언 좋아한다 는 positive로 예상됩니다
댓글 :방송 시간 변경 되며 방종 시간 시가 넘어가면 이상하다 싶을 정도 실력 떨어져 보일 있는데 이럴 면 시청자 사이 시 요정 왔다는 말 올라온다 는 positive로 예상됩니다
댓글 :본디 실력 보고 방송 즐겨 보는 많아서 헬 퍼 논란 자체 상당히 안 좋게 작용 열 혈 팬 몇몇 사라졌고 시청자 수도 많이 감소 상황 는 positive로 예상됩니다
댓글 :한편 게임 실력 가늠 일반 적 기준 클리어 여부 가지 고 따진다면 풍월 량 실력 오히려 평균 이상 이라고 말 는 positive로 예상됩니다
댓글 :어마어마한 실력 페북 영상 퍼가거 나 포우 페 먹는 경우 종종 는 positive로 예상됩니다
댓글 :종합 게임 스트리머 표방 하며 상반기 주로 게임 배틀 라운드 리그 오브 레 전드 외 스팀 게임 자주 편 게임 적 지능 및 실력 나쁘지 않으나 성격 급해서 지능 성 게임 이나 차분히 살펴보고 판단 해야 게임 쪽 약한 면 는 negative로 예상됩니다
댓글 :그래도 욕 반일 같은 게임 따라 크게 달라지는데 전체 적 으로는 무난 하게 잘 편 실력 비해 자존심 센 컨셉 주된 웃음 포인트 는 positive로 예상됩니다
댓글 :시즌 마지막 날인 시즌 세기말 솔로 랭크 게임 전설 되었다 는 negative로 예상됩니다
댓글 :승급 전 전 마지막 경기 브론즈 LP 승리 덕분 인지 승급 전 승 받고 시작 하였다 는 negative로 예상됩니다
댓글 :저 라 뎃 찬 처음 만나게 된 썰 즈 려 방송국 저 라 뎃 방송 즐겨 보는 고등학생 천상계 정글 러이다 는 positive로 예상됩니다
댓글 :와중 추리 부분 에서는 역시 나 헤맸다 는 positive로 예상됩니다
댓글 :영웅 모두 화려한 실력 

In [14]:
# 로지스틱 회귀 + Tf-idf


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(C=10))

])

pipeline.fit(X_train['sentence'], y_train)
pred = pipeline.predict(X_test['sentence'])
pred_probs = pipeline.predict_proba(X_test['sentence'])[:,1]

print('예측 정확도는 {0:.4f}, ROC_AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

cnt = 0
for review in X_test['sentence']:
    if pred[cnt] == 1:
        sentiment = 'positive'
    else:
        sentiment = 'negative'

    cnt += 1
    if cnt == 100:
        break

    print('댓글 :{0} 는 {1}로 예상됩니다'.format(review, sentiment))

예측 정확도는 0.7875, ROC_AUC는 0.8711
댓글 :챔피언 폭도 진 같은 챔피언 보다는 베인 이나 트위치 코그모 같은 AD 하이퍼 캐리 선호 하며 그것 아니라면 케이틀린 이나 자야 같은 챔피언 골라 라인 전 에서부터 압살 하려 챔피언 좋아한다 는 positive로 예상됩니다
댓글 :방송 시간 변경 되며 방종 시간 시가 넘어가면 이상하다 싶을 정도 실력 떨어져 보일 있는데 이럴 면 시청자 사이 시 요정 왔다는 말 올라온다 는 positive로 예상됩니다
댓글 :본디 실력 보고 방송 즐겨 보는 많아서 헬 퍼 논란 자체 상당히 안 좋게 작용 열 혈 팬 몇몇 사라졌고 시청자 수도 많이 감소 상황 는 positive로 예상됩니다
댓글 :한편 게임 실력 가늠 일반 적 기준 클리어 여부 가지 고 따진다면 풍월 량 실력 오히려 평균 이상 이라고 말 는 positive로 예상됩니다
댓글 :어마어마한 실력 페북 영상 퍼가거 나 포우 페 먹는 경우 종종 는 positive로 예상됩니다
댓글 :종합 게임 스트리머 표방 하며 상반기 주로 게임 배틀 라운드 리그 오브 레 전드 외 스팀 게임 자주 편 게임 적 지능 및 실력 나쁘지 않으나 성격 급해서 지능 성 게임 이나 차분히 살펴보고 판단 해야 게임 쪽 약한 면 는 negative로 예상됩니다
댓글 :그래도 욕 반일 같은 게임 따라 크게 달라지는데 전체 적 으로는 무난 하게 잘 편 실력 비해 자존심 센 컨셉 주된 웃음 포인트 는 positive로 예상됩니다
댓글 :시즌 마지막 날인 시즌 세기말 솔로 랭크 게임 전설 되었다 는 positive로 예상됩니다
댓글 :승급 전 전 마지막 경기 브론즈 LP 승리 덕분 인지 승급 전 승 받고 시작 하였다 는 negative로 예상됩니다
댓글 :저 라 뎃 찬 처음 만나게 된 썰 즈 려 방송국 저 라 뎃 방송 즐겨 보는 고등학생 천상계 정글 러이다 는 positive로 예상됩니다
댓글 :와중 추리 부분 에서는 역시 나 헤맸다 는 positive로 예상됩니다
댓글 :영웅 모두 화려한 실력 

In [3]:
# 형태소 분석버전

import nltk
from konlpy.tag import Okt
t=Okt()

content_text = ""

for sentence in sentence_df['sentence']:
    content_text = content_text + sentence + "\n"

print(content_text)

카오스  아키에이지      년     년 리그 오브 레전드      년 오버워치      년     년 배틀그라운드 등 그 해에 가장 인기를 끌었던 게임마다 파고들면 최상위권을 달성할 정도로 여러 게임을 잘해서 유명해졌다 
어떤 게임이든지 파고들기 시작하면 최상위권을 달성하며 엄청난 실력의 재능충 모습을 보여줬다  
게임 센스가 매우 뛰어난 편이며 화려한 플레이가 자주 나오는 편이다  
이때는 잿불거인이 나타나기 전으로 우디르가 고인챔이라는 인식이 강해 천상계에서 아무도 우디르를 쓰지 않았으나 우디르   판 승률     KDA    이라는 어마어마한 수치로 챌린저에 입성했다 
당연히 천상계에선 원탑 우디르 유저 
오버워치와 비슷한 FPS게임에 재능을 보이는 파카답게 솔로 스쿼드를 가리지 않고 엄청난 에임을 보여주고 있다  
롤 PD들이 파카 방송에 모여 스쿼드를 자주 하였는데 PD들 한두 명이  킬   킬을 하거나 빨리 죽어서 없더라도 멱살잡고 우승시켜주는 버스 기사 역할을 하였다 
플레이 스타일은 피지컬과 반응속도로 찍어누르는 재능충의 표본  
뛰어난 로지컬과 상황 판단력으로 각을 재고 들어가는 타입이 아니라 일단 몸을 던지며 최대한 공격적인 포지셔닝을 잡는다 
   의 상황에서도 본인의 피지컬을 신뢰하며 절대 빼지 않고 몸을 던진다 
챔피언 폭도 진 같은 챔피언보다는 베인이나 트위치  코그모 같은 AD하이퍼캐리를 선호하며 그것이 아니라면 케이틀린이나 자야 같은 챔피언을 골라 라인전에서부터 압살을 하려 하는 챔피언을 좋아한다 
딸피로 살아나오는 플레이가 일품 
캐리력이 높은 브루저 챔피언으로 라인전을 터트리고 미드  바텀 순으로 똥을 퍼트리는 전형적인 탑신병자의 캐리 시나리오를 선호하는 모습 
 영웅 모두 화려한 실력으로 노마이크에 솔로큐임에도 불구하고 시즌  경쟁전   승  무   패       로   점을 달성했다 
하지만 예전과 같은 플레이를 보여주지 못해 팟수들의 실망을 자아내었다 
오버워치와 비슷한 FPS게임에 재능을 보이는 파카답게 솔로 스쿼드를

In [4]:
# 형태소 분석

tokens_ko = t.morphs(content_text)

In [5]:
ko = nltk.Text(tokens_ko)
print(len(ko.tokens))
print(len(set(ko.tokens)))

11631
3105


In [46]:
#빈도수 확인 및 불용어 제거
ko.vocab().most_common(100)

[('을', 416),
 ('이', 287),
 ('실력', 258),
 ('를', 229),
 ('의', 209),
 ('에', 208),
 ('로', 134),
 ('들', 105),
 ('으로', 103),
 ('에서', 103),
 ('게임', 97),
 ('가', 95),
 ('은', 93),
 ('하는', 92),
 ('도', 90),
 ('는', 80),
 ('한', 80),
 ('그', 64),
 ('플레이', 60),
 ('때', 57),
 ('있다', 57),
 ('할', 54),
 ('적', 53),
 ('것', 51),
 ('년', 50),
 ('인', 49),
 ('등', 48),
 ('방송', 48),
 ('과', 48),
 ('정도', 47),
 ('전', 47),
 ('월', 46),
 ('했다', 41),
 ('와', 41),
 ('일', 41),
 ('수', 41),
 ('한다', 41),
 ('하고', 39),
 ('시즌', 39),
 ('이다', 39),
 ('잘', 33),
 ('못', 33),
 ('다', 32),
 ('나', 31),
 ('말', 30),
 ('모습', 29),
 ('까지', 28),
 ('선수', 28),
 ('중', 28),
 ('시청자', 28),
 ('실버', 28),
 ('롤', 27),
 ('위', 27),
 ('본인', 26),
 ('하며', 25),
 ('고', 25),
 ('에게', 25),
 ('최고', 25),
 ('해', 23),
 ('다른', 23),
 ('티어', 23),
 ('프로', 23),
 ('승', 22),
 ('\n', 21),
 ('자신', 21),
 ('때문', 21),
 ('라운드', 20),
 ('자주', 20),
 ('같은', 20),
 ('라는', 20),
 ('사람', 20),
 ('에는', 20),
 ('챔피언', 19),
 ('이나', 19),
 ('패', 19),
 ('이후', 19),
 ('팀', 19),
 ('엄청난', 18),
 ('매우', 18

In [6]:
stop_words = ['을', '를','의','이','에','로','가','들','으로','은','도','에서','했다','것','인','는','와','있다','때','과','그','하고','한다','년','수','이다','하는','한','할','등','월','일','까지','에게','다']

tokens_ko = [each_word for each_word in tokens_ko
                if each_word not in stop_words]

ko = nltk.Text(tokens_ko)
ko.vocab().most_common(100)

[('실력', 258),
 ('게임', 97),
 ('플레이', 60),
 ('적', 53),
 ('방송', 48),
 ('정도', 47),
 ('전', 47),
 ('시즌', 39),
 ('잘', 33),
 ('못', 33),
 ('나', 31),
 ('말', 30),
 ('모습', 29),
 ('선수', 28),
 ('중', 28),
 ('시청자', 28),
 ('실버', 28),
 ('롤', 27),
 ('위', 27),
 ('본인', 26),
 ('하며', 25),
 ('고', 25),
 ('최고', 25),
 ('해', 23),
 ('다른', 23),
 ('티어', 23),
 ('프로', 23),
 ('승', 22),
 ('\n', 21),
 ('자신', 21),
 ('때문', 21),
 ('라운드', 20),
 ('자주', 20),
 ('같은', 20),
 ('라는', 20),
 ('사람', 20),
 ('에는', 20),
 ('챔피언', 19),
 ('이나', 19),
 ('패', 19),
 ('이후', 19),
 ('팀', 19),
 ('엄청난', 18),
 ('매우', 18),
 ('게', 18),
 ('컬', 18),
 ('저', 18),
 ('영상', 18),
 ('상대', 18),
 ('컨트롤', 18),
 ('많이', 18),
 ('리그', 17),
 ('오브', 17),
 ('여러', 17),
 ('천상계', 17),
 ('챌', 17),
 ('하지만', 17),
 ('서', 17),
 ('클리어', 17),
 ('배틀', 16),
 ('달성', 16),
 ('시작', 16),
 ('편이', 16),
 ('킬', 16),
 ('점', 16),
 ('솔', 16),
 ('편', 16),
 ('골드', 16),
 ('핵', 16),
 ('듀오', 16),
 ('하면', 15),
 ('유저', 15),
 ('피지', 15),
 ('만', 15),
 ('된', 15),
 ('있는', 15),
 ('권', 14),
 ('라인', 14),
 ('

In [7]:
i = 0
for sentence in sentence_df['sentence']:
    morph_sentence = t.morphs(sentence)
    clean_sentence = [each_word for each_word in morph_sentence
                if each_word not in stop_words]
    new_sentence = " ".join(clean_sentence)
    new_sentence = re.sub(pattern="피지 컬", repl="피지컬", string=new_sentence)
    new_sentence = re.sub(pattern="상위 권", repl="상위권", string=new_sentence)
    sentence_df['sentence'][i] = new_sentence
    i += 1

sentence_df.head(10)

,sentence,lable
0,카오스 아키에이지 리그 오브 레전드 오버 워치 배틀 라운드 해 가장 인기 끌었던 게...,1
1,어떤 게임 이든지 파고들기 시작 하면 최 상위권 달성 하며 엄청난 실력 재능 충 모...,1
2,게임 센스 매우 뛰어난 편이 며 화려한 플레이 자주 나오는 편이,1
3,잿불 거인 나타나기 전 우디르 고인 챔이라는 인식 강해 천상계 아무 우디르 쓰지 않...,1
4,당연히 천상계 에선 원 탑 우디르 유저,1
5,오버 워치 비슷한 FPS 게임 재능 보이는 파카 답 게 솔로 스쿼드 가리지 않고 엄...,1
6,롤 PD 들이 파카 방송 모여 스쿼드 자주 하였는데 PD 한두 명 킬 킬 하 거나 ...,1
7,플레이 스타일 피지컬 반응속도 찍어 누르는 재능 충의 표본,1
8,뛰어난 로지 컬 상황 판단력 각 재고 들어가는 타입 아니라 일단 몸 던지며 최대한 ...,1
9,상황 에서도 본인 피지컬 신뢰 하며 절대 빼지 않고 몸 던진다,1


In [15]:
# 모델 저장

import pickle

with open('pipe_full-tfidf-2.dat', 'wb') as fp:
    pickle.dump(pipeline, fp)

In [3]:
# 모델 불러오기
import pickle

with open('pipe_tfidf-2.dat', 'rb') as fp:
        pipeline = pickle.load(fp)

In [105]:
# 새로운 문장 예측함수

def using_model(string):
    import numpy as np
    with open('pipe_tfidf.dat', 'rb') as fp:
        pipeline = pickle.load(fp)

    text = [string]
    

    # 예측 스코어
    r1 = np.max(pipeline.predict_proba(text) * 100)
    
    # 예측 결과
    r2 = pipeline.predict(text)[0]
    

    if r2 == 1 :
        print('실력 좋다')
    else :
        print('실력 나쁘다')

    print('스코어 : %.3f '% r1)


using_model('진짜 존나 못한다')

실력 나쁘다
스코어 : 56.659 
